In [0]:
%run "/Workspace/Users/sundarasandeepteja@gmail.com/E-Commerce Analytics Medallion Architecture with GenAI/config/project_config"

In [0]:
# Databricks notebook source
# ======================================
# GOLD LAYER: DIM_PRODUCTS (PRODUCT DIMENSION)
# ======================================

# MAGIC %run ../config/project_config

from pyspark.sql import functions as F
from pyspark.sql.window import Window

print("🥇 GOLD LAYER: Creating dim_products")
print("=" * 60)

# ======================================
# LOAD SILVER DATA
# ======================================
print("\n📥 Loading Silver products...")

silver_products = spark.table(SILVER_PRODUCTS_TABLE)
print(f"  Silver records: {silver_products.count():,}")

# Load product ratings summary if available
try:
    product_ratings = spark.table(f"{SILVER_DB}.product_ratings_summary")
    has_ratings = True
    print(f"  Ratings summary: {product_ratings.count():,}")
except:
    has_ratings = False
    print("  ⚠️ No ratings summary found, proceeding without")

# ======================================
# CREATE SURROGATE KEY
# ======================================
print("\n🔑 Creating surrogate keys...")

# Create sequential surrogate key
window = Window.orderBy("product_id")

dim_products = silver_products \
    .withColumn("product_key", F.row_number().over(window))

print("  ✅ Surrogate keys generated")

# ======================================
# JOIN RATING METRICS (IF AVAILABLE)
# ======================================
if has_ratings:
    print("\n🔗 Joining rating metrics...")
    
    dim_products = dim_products.join(
        product_ratings.select(
            "product_id",
            "total_ratings",
            "total_reviews",
            F.col("avg_rating").alias("calculated_avg_rating"),
            "positive_rate",
            "sentiment_score",
            "rating_confidence"
        ),
        on="product_id",
        how="left"
    ).fillna({
        "total_ratings": 0,
        "total_reviews": 0,
        "positive_rate": 0,
        "sentiment_score": 0,
        "rating_confidence": "No Data"
    })
    
    print("  ✅ Rating metrics joined")

# ======================================
# ADD DIMENSION ATTRIBUTES
# ======================================
print("\n➕ Adding dimension attributes...")

dim_products = dim_products \
    .withColumn("product_rank_by_price",
        F.dense_rank().over(Window.partitionBy("category").orderBy(F.desc("price")))
    ) \
    .withColumn("product_rank_by_rating",
        F.dense_rank().over(Window.partitionBy("category").orderBy(F.desc("avg_rating")))
    ) \
    .withColumn("is_top_rated",
        F.when(F.col("avg_rating") >= 4.5, True).otherwise(False)
    ) \
    .withColumn("is_premium",
        F.when(F.col("price_tier").isin("Premium", "Luxury", "Ultra-Luxury"), True).otherwise(False)
    ) \
    .withColumn("is_high_margin",
        F.when(F.col("profit_margin") >= 40, True).otherwise(False)
    ) \
    .withColumn("_loaded_at", F.current_timestamp())

# ======================================
# SELECT FINAL COLUMNS
# ======================================
print("\n📋 Selecting final columns...")

final_columns = [
    # Keys
    "product_key",
    "product_id",
    
    # Product details
    "product_name",
    "category",
    "subcategory",
    "brand",
    
    # Pricing
    "price",
    "cost",
    "profit_margin",
    "profit_amount",
    "price_tier",
    "margin_category",
    
    # Ratings
    "avg_rating",
    "rating_category",
    
    # Inventory
    "stock_quantity",
    "stock_status",
    "is_in_stock",
    "is_available",
    
    # Flags
    "is_top_rated",
    "is_premium",
    "is_high_margin",
    
    # Rankings
    "product_rank_by_price",
    "product_rank_by_rating",
    
    # Metadata
    "_loaded_at"
]

# Add rating columns if available
if has_ratings:
    final_columns.extend([
        "total_ratings",
        "total_reviews", 
        "positive_rate",
        "sentiment_score",
        "rating_confidence"
    ])

dim_products_final = dim_products.select(final_columns)

# ======================================
# WRITE TO GOLD
# ======================================
print("\n💾 Writing to Gold layer...")

dim_products_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(GOLD_DIM_PRODUCTS_TABLE)

print(f"  ✅ Created: {GOLD_DIM_PRODUCTS_TABLE}")

# Optimize
spark.sql(f"OPTIMIZE {GOLD_DIM_PRODUCTS_TABLE} ZORDER BY (category, brand)")
print("  ✅ Optimized with ZORDER")

# ======================================
# VERIFY AND SUMMARIZE
# ======================================
print("\n📊 Summary Statistics:")

final_count = spark.table(GOLD_DIM_PRODUCTS_TABLE).count()
print(f"  Total products: {final_count:,}")

# Category distribution
print("\n📊 Products by Category:")
spark.table(GOLD_DIM_PRODUCTS_TABLE) \
    .groupBy("category") \
    .agg(
        F.count("*").alias("products"),
        F.round(F.avg("price"), 2).alias("avg_price"),
        F.round(F.avg("profit_margin"), 2).alias("avg_margin")
    ) \
    .orderBy(F.desc("products")) \
    .show()

# Price tier distribution
print("📊 Products by Price Tier:")
spark.table(GOLD_DIM_PRODUCTS_TABLE) \
    .groupBy("price_tier") \
    .agg(F.count("*").alias("count")) \
    .orderBy(F.desc("count")) \
    .show()

print("\n🥇 DIM_PRODUCTS COMPLETE!")